In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


In [2]:
datasig=pd.read_csv("sig_many_values.csv")
databkg=pd.read_csv("bkg_many_values.csv")

In [3]:
datasig.shape

(67263, 57)

In [4]:
databkg.shape

(84499, 57)

In [5]:
databkg.iloc[2:100,4];

In [6]:
datasig["label"]=1
databkg["label"]=2

In [7]:
databkg.shape

(84499, 58)

In [8]:
data2=datasig.append(databkg,ignore_index=True)

In [9]:
data2.shape

(151762, 58)

In [10]:
X,Y=data2.iloc[:,:-3],data2.iloc[:,-1]
weights=data2.iloc[:,-2]

In [11]:
import xgboost as xgb


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=7)

In [13]:
xg_class= xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 8, alpha = 10, n_estimators = 10)
#xg_class= xgb.XGBClassifier()


xg_class= XGBClassifier()

In [14]:
xgmodel=xg_class.fit(X_train, y_train)
preds=xg_class.predict(X_test)

In [15]:
print(xg_class);

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
       nthread=None, objective='reg:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)


In [16]:
predictions = [round(value) for value in preds]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 60.56%


In [17]:
preds

array([1, 2, 2, ..., 2, 2, 2])